##Required Packages

In [ ]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image
import os
from PIL import Image
import numpy as np
from numpy import dot
from numpy.linalg import norm
import math

# Inference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
MAIN_PATH = "/content/drive/MyDrive/학교 수업/2023 4학년 1학기/졸업프로젝트/RealTimeFaceRecognition/"

## Data

In [ ]:
import pickle
with open(MAIN_PATH+'final_data/crop_30_resize.pkl','rb') as f:
  test_crop_data = pickle.load(f)

In [ ]:
with open(MAIN_PATH+'final_data/crop_standard_resize_30.pkl','rb') as f:
  test_standard_array = pickle.load(f)

In [ ]:
with open(MAIN_PATH+'final_data/image_label_30.pkl','rb') as f:
  test_crop_label = pickle.load(f)

with open(MAIN_PATH+'final_data/standard_label_list30.pkl','rb') as f:
  test_standard_label = pickle.load(f)

###Model

In [ ]:
from numpy import dot
from numpy.linalg import norm

# 유사도 정의 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# 유클리드 거리
def euclidean(A, B):
  temp = A - B
  dist = np.sqrt(np.dot(temp.T, temp))
  return dist

## \_\_init__.py

In [ ]:
# from .iresnet import iresnet18, iresnet34, iresnet50, iresnet100, iresnet200
def get_model(name, **kwargs):
    # resnet
    if name == "r18":
        return iresnet18(False, **kwargs)
    elif name == "r34":
        return iresnet34(False, **kwargs)
    elif name == "r50":
        return iresnet50(False, **kwargs)
    elif name == "r100":
        return iresnet100(False, **kwargs)
    elif name == "r200":
        return iresnet200(False, **kwargs)
    else:
        raise ValueError()

## iresnet.py

In [ ]:
import torch
from torch import nn
from torch.utils.checkpoint import checkpoint

__all__ = ['iresnet34']
using_ckpt = False

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=dilation,
                     groups=groups,
                     bias=False,
                     dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)


class IBasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None,
                 groups=1, base_width=64, dilation=1):
        super(IBasicBlock, self).__init__()
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        self.bn1 = nn.BatchNorm2d(inplanes, eps=1e-05,)
        self.conv1 = conv3x3(inplanes, planes)
        self.bn2 = nn.BatchNorm2d(planes, eps=1e-05,)
        self.prelu = nn.PReLU(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn3 = nn.BatchNorm2d(planes, eps=1e-05,)
        self.downsample = downsample
        self.stride = stride

    def forward_impl(self, x):
        identity = x
        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return out

    def forward(self, x):
        if self.training and using_ckpt:
            return checkpoint(self.forward_impl, x)
        else:
            return self.forward_impl(x)


class IResNet(nn.Module):
    fc_scale = 7 * 7
    def __init__(self,
                 block, layers, dropout=0, num_features=512, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None, fp16=False):
        super(IResNet, self).__init__()
        self.extra_gflops = 0.0
        self.fp16 = fp16
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes, eps=1e-05)
        self.prelu = nn.PReLU(self.inplanes)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=2)
        self.layer2 = self._make_layer(block,
                                       128,
                                       layers[1],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block,
                                       256,
                                       layers[2],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block,
                                       512,
                                       layers[3],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.bn2 = nn.BatchNorm2d(512 * block.expansion, eps=1e-05,)
        self.dropout = nn.Dropout(p=dropout, inplace=True)
        self.fc = nn.Linear(512 * block.expansion * self.fc_scale, num_features)
        self.features = nn.BatchNorm1d(num_features, eps=1e-05)
        nn.init.constant_(self.features.weight, 1.0)
        self.features.weight.requires_grad = False

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, 0, 0.1)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, IBasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion, eps=1e-05, ),
            )
        layers = []
        layers.append(
            block(self.inplanes, planes, stride, downsample, self.groups,
                  self.base_width, previous_dilation))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(self.inplanes,
                      planes,
                      groups=self.groups,
                      base_width=self.base_width,
                      dilation=self.dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        with torch.cuda.amp.autocast(self.fp16):
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.prelu(x)
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.bn2(x)
            x = torch.flatten(x, 1)
            x = self.dropout(x)
        x = self.fc(x.float() if self.fp16 else x)
        x = self.features(x)
        return x


def _iresnet(arch, block, layers, pretrained, progress, **kwargs):
    model = IResNet(block, layers, **kwargs)
    if pretrained:
        raise ValueError()
    return model

def iresnet18(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet18', IBasicBlock, [2, 2, 2, 2], pretrained,
                    progress, **kwargs)

def iresnet34(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet34', IBasicBlock, [3, 4, 6, 3], pretrained,
                    progress, **kwargs)


def iresnet50(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet50', IBasicBlock, [3, 4, 14, 3], pretrained,
                    progress, **kwargs)


def iresnet50(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet50', IBasicBlock, [3, 4, 14, 3], pretrained,
                    progress, **kwargs)

def iresnet100(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet100', IBasicBlock, [3, 13, 30, 3], pretrained,
                    progress, **kwargs)

# inference

In [ ]:
import cv2
import numpy as np
import torch

# from backbones import get_model

@torch.no_grad()
def inference_2(name, array, device):
    tic = time.time()
    img = cv2.cvtColor(array, cv2.COLOR_BGR2RGB) # 이미지의 색상 공간을 변환
    img = np.transpose(img, (2, 0, 1)) # PyTorch에서 사용하기 위해 배열의 차원 바꾸기
    img = torch.from_numpy(img).unsqueeze(0).float() # img를 PyTorch 텐서로 변환하고, 0번째 차원을 하나 늘리기 (배치 차원을 추가하는 데 사용됨)
    img.div_(255).sub_(0.5).div_(0.5) # 이미지 픽셀 값을 255로 나누고(0~1), 0.5를 빼서(-0.5~0.5), 다시 0.5로 나눈다(-1~1).
    net.eval() # 모델을 평가 모드로 설정
    feat = net(img.to(device)).cpu().numpy() # img를 모델에 전달하여 예측을 실행하고, 결과를 numpy 배열로 변환한다.
    print("net forward time: {:.4f}".format(time.time() - tic))

    return feat

# 4. 결과 확인

In [ ]:
from tqdm import tqdm
import time

In [ ]:
network_recog = 'r18'
net = get_model(network_recog, fp16=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_dict = torch.load(MAIN_PATH+'final_data/fine_tuning_result/r18_Combined_sgd_20ep_resize_batch32/mydata_onegpu/model.pt'
, map_location=device)
net.load_state_dict(pretrained_dict) # 저장된 모델의 가중치 로드
net = net.to(device) # 모델을 device로 이동
net.eval()
print(device)

cuda


In [ ]:
#기준 얼굴 feature vector 생성
standard_dic = []

for standard_path in tqdm(test_standard_array):

  feat = inference_2(network_recog, standard_path, device)

  #print(standard_path)
  standard_dic.append(feat)

100%|██████████| 30/30 [00:00<00:00, 159.72it/s]

net forward time: 0.0115
net forward time: 0.0071
net forward time: 0.0060
net forward time: 0.0061
net forward time: 0.0058
net forward time: 0.0059
net forward time: 0.0060
net forward time: 0.0058
net forward time: 0.0063
net forward time: 0.0058
net forward time: 0.0058
net forward time: 0.0058
net forward time: 0.0058
net forward time: 0.0058
net forward time: 0.0057
net forward time: 0.0054
net forward time: 0.0080
net forward time: 0.0057
net forward time: 0.0070
net forward time: 0.0058
net forward time: 0.0058
net forward time: 0.0065
net forward time: 0.0058
net forward time: 0.0057
net forward time: 0.0057
net forward time: 0.0058
net forward time: 0.0059
net forward time: 0.0058
net forward time: 0.0054
net forward time: 0.0045


In [ ]:
#모든 파일 feature vector 생성
test_file = []

for image in tqdm(test_crop_data):
  feat = inference_2(network_recog, image, device)
  test_file.append(feat)

  0%|          | 19/6870 [00:00<00:38, 178.32it/s]

net forward time: 0.0095
net forward time: 0.0048
net forward time: 0.0054
net forward time: 0.0047
net forward time: 0.0044
net forward time: 0.0070
net forward time: 0.0048
net forward time: 0.0048
net forward time: 0.0042
net forward time: 0.0048
net forward time: 0.0043
net forward time: 0.0046
net forward time: 0.0049
net forward time: 0.0047
net forward time: 0.0051
net forward time: 0.0045
net forward time: 0.0045
net forward time: 0.0076
net forward time: 0.0074
net forward time: 0.0054
net forward time: 0.0064
net forward time: 0.0116
net forward time: 0.0047
net forward time: 0.0047
net forward time: 0.0047
net forward time: 0.0045
net forward time: 0.0046
net forward time: 0.0047
net forward time: 0.0045
net forward time: 0.0048
net forward time: 0.0047
net forward time: 0.0048
net forward time: 0.0045
net forward time: 0.0056
net forward time: 0.0046
net forward time: 0.0047


  1%|          | 59/6870 [00:00<00:36, 188.83it/s]

net forward time: 0.0072
net forward time: 0.0058
net forward time: 0.0046
net forward time: 0.0045
net forward time: 0.0047
net forward time: 0.0045
net forward time: 0.0046
net forward time: 0.0045
net forward time: 0.0045
net forward time: 0.0044
net forward time: 0.0040
net forward time: 0.0042
net forward time: 0.0040
net forward time: 0.0043
net forward time: 0.0040
net forward time: 0.0042
net forward time: 0.0056
net forward time: 0.0048
net forward time: 0.0042
net forward time: 0.0039
net forward time: 0.0041
net forward time: 0.0039
net forward time: 0.0041
net forward time: 0.0066
net forward time: 0.0042
net forward time: 0.0041
net forward time: 0.0040
net forward time: 0.0043
net forward time: 0.0043
net forward time: 0.0041
net forward time: 0.0043
net forward time: 0.0041
net forward time: 0.0041
net forward time: 0.0044
net forward time: 0.0042
net forward time: 0.0042
net forward time: 0.0040
net forward time: 0.0042
net forward time: 0.0043
net forward time: 0.0040


  2%|▏         | 104/6870 [00:00<00:32, 206.79it/s]

...

net forward time: 0.0070
net forward time: 0.0052
net forward time: 0.0042
net forward time: 0.0040
net forward time: 0.0040
net forward time: 0.0040
net forward time: 0.0042
net forward time: 0.0042
net forward time: 0.0044
net forward time: 0.0041
net forward time: 0.0045
net forward time: 0.0041
net forward time: 0.0075
net forward time: 0.0052
net forward time: 0.0043
net forward time: 0.0043
net forward time: 0.0044
net forward time: 0.0040
net forward time: 0.0043
net forward time: 0.0041
net forward time: 0.0042
net forward time: 0.0044
net forward time: 0.0043
net forward time: 0.0044
net forward time: 0.0044
net forward time: 0.0057
net forward time: 0.0041
net forward time: 0.0042
net forward time: 0.0041
net forward time: 0.0044
net forward time: 0.0040
net forward time: 0.0041
net forward time: 0.0042
net forward time: 0.0045
net forward time: 0.0057
net forward time: 0.0059
net forward time: 0.0043
net forward time: 0.0043
net forward time: 0.0044
net forward time: 0.0057


 99%|█████████▉| 6835/6870 [00:33<00:00, 213.48it/s]

net forward time: 0.0078
net forward time: 0.0063
net forward time: 0.0054
net forward time: 0.0053
net forward time: 0.0043
net forward time: 0.0042
net forward time: 0.0040
net forward time: 0.0042
net forward time: 0.0040
net forward time: 0.0040
net forward time: 0.0040
net forward time: 0.0040
net forward time: 0.0070
net forward time: 0.0044
net forward time: 0.0040
net forward time: 0.0040
net forward time: 0.0042
net forward time: 0.0040
net forward time: 0.0043
net forward time: 0.0040
net forward time: 0.0040
net forward time: 0.0041
net forward time: 0.0042
net forward time: 0.0044
net forward time: 0.0046
net forward time: 0.0044
net forward time: 0.0042
net forward time: 0.0041
net forward time: 0.0036
net forward time: 0.0042
net forward time: 0.0042
net forward time: 0.0040
net forward time: 0.0046
net forward time: 0.0041
net forward time: 0.0040
net forward time: 0.0066
net forward time: 0.0050
net forward time: 0.0040
net forward time: 0.0041
net forward time: 0.0041


100%|██████████| 6870/6870 [00:33<00:00, 203.07it/s]

net forward time: 0.0070
net forward time: 0.0045
net forward time: 0.0043
net forward time: 0.0040
net forward time: 0.0067
net forward time: 0.0040
net forward time: 0.0039
net forward time: 0.0042
net forward time: 0.0044
net forward time: 0.0041
net forward time: 0.0041
net forward time: 0.0047
net forward time: 0.0127
net forward time: 0.0042
net forward time: 0.0043
net forward time: 0.0049
net forward time: 0.0040
net forward time: 0.0043
net forward time: 0.0040
net forward time: 0.0041
net forward time: 0.0042
net forward time: 0.0044
net forward time: 0.0043
net forward time: 0.0042
net forward time: 0.0045


In [ ]:
with open(MAIN_PATH+'final_data/feature_vector/r18_Combined_sgd_20ep_resize_batch32.pkl', 'wb') as f:
     pickle.dump(test_file, f)

In [ ]:
threshold = 0.05

#cosine similarity
predict_label = []
similarity_score = []
for i in range(len(test_file)):
  test_similarity = []
  #각 기준파일과의 cosine similarity 계산 => 유사도가 가장 높은 이미지가 0.7 이상이면 label로 사용
  for j in range(len(standard_dic)):
    sim = cos_sim(np.array(test_file[i]), np.array(standard_dic[j]).reshape(-1,1))
    test_similarity.append(sim)

  test_similarity = np.array(test_similarity)
  max_value = np.max(test_similarity)
  similarity_score.append(max_value)

  if max_value > threshold:
      index = np.argmax(test_similarity)
      max_name = test_standard_label[index]

  else:
      max_name = 'Unknown'

  # index = np.argmax(test_similarity)
  # max_name = standard_name[index]

  predict_label.append(max_name)

## Accuracy

In [ ]:
# 다중 클래스 평가 지표 계산
accuracy = accuracy_score(test_crop_label, predict_label)
precision = precision_score(test_crop_label, predict_label, average='macro')
recall = recall_score(test_crop_label, predict_label, average='macro')
f1 = f1_score(test_crop_label, predict_label, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8259097525473071
Precision: 0.8463118725828396
Recall: 0.8259097525473073
F1 Score: 0.825298274911059


In [ ]:
import unicodedata
num = 0
for i in range(len(predict_label)):
  label_predict = predict_label[i]
  label_actual = str(list(test_crop_label)[i])
  if label_predict == label_actual:
     num+=1
  else:
    #print(f'\n index: {i} true:{image_list_label[i]} predict:{predict_label[i]}')
    #img = cv2.imread(test_dataset[i])
    #plt.imshow(img[:,:,::-1])
    #plt.show()
    pass
print(num)
print("Accuracy: ", num/len(predict_label))

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



5810
Accuracy:  0.8457059679767104
Accuracy: 0.8457059679767104
Precision: 0.8581491797209588
Recall: 0.8457059679767104
F1 Score: 0.8463117790157936


In [ ]:
for i,j,s in zip(test_crop_label, predict_label, similarity_score):
  print(i, j, s)

19101512 19101512 0.60463715
19101512 19101512 0.59332603
19101512 19101512 0.37351537
19101512 19101512 0.6039004
19101512 19101141 0.38012707
19101512 19101512 0.5445757
19101512 19101512 0.58415705
19101512 19101512 0.5959807
19101512 19101512 0.55299354
19101512 19101512 0.5551803
19101512 19101512 0.38687703
19101512 19101512 0.50550646
19101512 19101512 0.93395346
19101512 19101513 0.3013777
19101512 19101512 0.5038746
19101512 19101512 0.74118394
19101512 19101512 0.6039084
19101512 19101512 0.55722225
19101512 19101512 0.44589546
19101512 19101512 0.34331155
19101512 19101512 0.801992
19101512 19101512 0.6757653
19101512 19101512 0.61482155
19101512 19101512 0.50151145
19101512 19101512 0.44904262
19101512 19101512 0.49924344
19101512 19101512 0.65935755
19101512 19101512 0.67779046
19101512 19101512 0.5530517
19101512 19101512 0.42153642
19101512 19101512 0.33940297
19101512 19100812 0.39254907
19101512 19101512 0.5140659
19101512 19101512 0.3984738
19101512 19101512 0.3665472

KeyboardInterrupt: ignored